# 13_encuestas_01 Fix missing data

While inspecting manualy drawn routes (__'RUTAS TOTAL.shp'__), one row was found to have missing values (Poll with code DP1V008).

This notebook finds the missing value for that poll by looking in the original Google Forms poll csv (__'Encuesta_caminata(Guadalajara)(1-553).csv'__) and adds the missing data to poll code DP1V008.

* __Output: preliminar_routes.shp__

In [1]:
#Autopep8

## Import libraries

In [2]:
first_folder_path = '../../../'

In [3]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import random
import scipy.stats as stats

import os
import sys
module_path = os.path.abspath(os.path.join(first_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Load data (Google Forms polls)

In [4]:
surveys_file = 'Encuesta_caminata(Guadalajara)(1-553).csv'
surveys = pd.read_csv(first_folder_path + f"data/processed/volvo/encuestas/01_encuestas_google_forms/{surveys_file}")

# Show
print(surveys.shape)
surveys.head(2)

(553, 65)


,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,3,4,5,6,7,8,9,10,11,12
0,1,6/28/24 12:24:50,6/28/24 12:25:46,anonymous,NaN,NaN,DS,6/25/2024,001,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
1,2,6/28/24 12:25:58,6/28/24 12:28:18,anonymous,NaN,NaN,DS,6/25/2024,002,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


In [5]:
surveys.Encuestador.unique()

array(['DS', 'JM', 'DP', 'DP ', '049', 'MV', 'RV', 'EE1V002', 'EE1V004',
       'EE1V001', 'EE1V005', 'EE1V003', 'Jesus Corona', 'Jesús Corona',
       'rv', 'MeV'], dtype=object)

## Load data (Manually drawn lines)

In [9]:
routes_file = 'RUTAS TOTAL.shp'
routes = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/rutas_total_antes_de_complementarias/{routes_file}")

# Show
print(routes.shape)
routes.head(2)

(184, 72)


,fid,Código,layer,path,Codigo_2,field_1,field_2,field_3,field_4,field_5,...,field_59,field_60,field_61,field_62,field_63,field_64,field_65,field_66,field_67,geometry
0,2,DP3V002,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,DP3002,252,9/19/24 18:40:07,9/19/24 18:46:58,anonymous,None,...,De acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,"LINESTRING (660511.087 2283608.716, 660443.453..."
1,5,DP3V005,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,DP3005,255,9/19/24 18:50:40,9/19/24 18:57:14,anonymous,None,...,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,En desacuerdo,De acuerdo,De acuerdo,"LINESTRING (660562.586 2283275.252, 660591.611..."


In [7]:
routes.columns

Index(['fid', 'Código', 'layer', 'path', 'Codigo_2', 'field_1', 'field_2',
       'field_3', 'field_4', 'field_5', 'field_6', 'field_7', 'field_8',
       'field_9', 'field_10', 'field_11', 'field_13', 'field_14', 'field_15',
       'field_16', 'field_17', 'field_18', 'field_19', 'field_20', 'field_21',
       'field_22', 'field_23', 'field_24', 'field_25', 'field_26', 'field_27',
       'field_28', 'field_29', 'field_30', 'field_31', 'field_32', 'field_33',
       'field_34', 'field_35', 'field_36', 'field_37', 'field_38', 'field_39',
       'field_40', 'field_41', 'field_42', 'field_43', 'field_44', 'field_45',
       'field_46', 'field_47', 'field_48', 'field_49', 'field_50', 'field_51',
       'field_52', 'field_53', 'field_54', 'field_55', 'field_56', 'field_57',
       'field_58', 'field_59', 'field_60', 'field_61', 'field_62', 'field_63',
       'field_64', 'field_65', 'field_66', 'field_67', 'geometry'],
      dtype='object')

In [8]:
dict(routes.dtypes)

{'fid': dtype('int64'),
 'Código': dtype('O'),
 'layer': dtype('O'),
 'path': dtype('O'),
 'Codigo_2': dtype('O'),
 'field_1': dtype('O'),
 'field_2': dtype('O'),
 'field_3': dtype('O'),
 'field_4': dtype('O'),
 'field_5': dtype('O'),
 'field_6': dtype('O'),
 'field_7': dtype('O'),
 'field_8': dtype('O'),
 'field_9': dtype('O'),
 'field_10': dtype('O'),
 'field_11': dtype('O'),
 'field_13': dtype('O'),
 'field_14': dtype('O'),
 'field_15': dtype('O'),
 'field_16': dtype('O'),
 'field_17': dtype('O'),
 'field_18': dtype('O'),
 'field_19': dtype('O'),
 'field_20': dtype('O'),
 'field_21': dtype('O'),
 'field_22': dtype('O'),
 'field_23': dtype('O'),
 'field_24': dtype('O'),
 'field_25': dtype('O'),
 'field_26': dtype('O'),
 'field_27': dtype('O'),
 'field_28': dtype('O'),
 'field_29': dtype('O'),
 'field_30': dtype('O'),
 'field_31': dtype('O'),
 'field_32': dtype('O'),
 'field_33': dtype('O'),
 'field_34': dtype('O'),
 'field_35': dtype('O'),
 'field_36': dtype('O'),
 'field_37': dtype(

## Show missing data (Found while exploring file in GIS)

In [9]:
routes.loc[routes.field_1.isna()]

,fid,Código,layer,path,Codigo_2,field_1,field_2,field_3,field_4,field_5,...,field_59,field_60,field_61,field_62,field_63,field_64,field_65,field_66,field_67,geometry
31,1008,DP1V008,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,DP1008,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"LINESTRING (668198.451 2288707.673, 668261.870..."


## Find missing poll data in Google Forms polls (Poll DP1V008)

In [10]:
# Look for rows whose data build code DP1V008
encuestador_lst = ['DP', 'DP ']
missing_data = surveys.loc[(surveys.Encuestador.isin(encuestador_lst))&(surveys["Territorio de análisis"]=='TA_Providencia')&(surveys["Encuesta #"]=='008')].copy()

# Show
missing_data

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,3,4,5,6,7,8,9,10,11,12
140,141,7/15/24 14:47:07,7/15/24 14:53:13,anonymous,NaN,NaN,DP,6/28/2024,008,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


## Rename cols

In [11]:
missing_data.rename(columns={'ID': 'field_1',
                             'Hora de inicio': 'field_2',
                             'Hora de finalización':'field_3',
                             'Correo electrónico':'field_4',
                             'Nombre':'field_5',
                             'Hora de la última modificación':'field_6',
                             'Encuestador':'field_7',
                             'Fecha de realización':'field_8',
                             'Encuesta #':'field_9',
                             #field_10 es el Territorio de análisis en número
                             'Territorio de análisis':'field_11',
                             #field_12 no existe
                             'Lugar donde se realizó la encuesta':'field_13',
                             'Género':'field_14',
                             'Edad (años)':'field_15',
                             'Estrato socioeconómico (NO APLICA)':'field_16',
                             'Ocupación principal':'field_17',
                             'Nivel educativo completado':'field_18',
                             '¿Poseen alguno de los siguientes vehículos en su vivienda?':'field_19',
                             'Indique su lugar de residencia (Municipio/comuna/barrio)':'field_20',
                             '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?':'field_21',
                             '¿Cómo suple las necesidades de equipamientos educativos?':'field_22',
                             '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?':'field_23',
                             '¿Cuáles?':'field_24',
                             '¿Hacia qué tipo de equipamiento/sitio de interés se dirige?':'field_25',
                             '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)': 'field_26',
                             'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando el mismo modo de transporte?': 'field_27',
                             'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando la caminata como modo de transporte? (CAMINATA)': 'field_28',
                             'Debido a la lejanía del equipamiento X': 'field_29',
                             'Debido a la ausencia de andenes': 'field_30',
                             'Porque no me siento seguro/a frente a otros vehículos': 'field_31',
                             'Porque no me siento seguro/a frente a posibles delitos en mi trayecto': 'field_32',
                             'Debido al esfuerzo que debo hacer a causa de la pendiente de las calles': 'field_33',
                             'Debido a que no me siento cómodo/a por el calor': 'field_34',
                             'Debido a que siento que los andenes muy saturados de gente': 'field_35',
                             'Debido a que el paisaje a lo largo del recorrido no es agradable': 'field_36',
                             'La cercanía del equipamiento X': 'field_37',
                             'La presencia de andenes': 'field_38',
                             'Sentirme seguro/a frente a otros vehículos': 'field_39',
                             'Sentirme seguro/a frente a posibles delitos en mi trayecto': 'field_40',
                             'La posibilidad de hacer múltiples vueltas/trámites en mi recorrido': 'field_41',
                             'La posibilidad de poder tomar desvíos y hacer múltiples trayectos en mi recorrido': 'field_42',
                             'Sentirme cómodo/a por la sombra generada por la vegetación': 'field_43',
                             'Sentirme cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido': 'field_44',
                             'Que el paisaje sea agradable': 'field_45',
                             '¿Se encuentra usted en alguna condición de discapacidad?': 'field_46',
                             'Indique cuál de las siguientes condiciones de discapacidad presenta': 'field_47',
                             'Indique el lugar donde inició de su recorrido lo más preciso posible (Punto de referencia, intersección más cercana)': 'field_48',
                             'Indique la hora de salida de su lugar de origen de recorrido (formato militar)': 'field_49',
                             'Ingrese los minutos de salida': 'field_50',
                             'Indique el lugar de destino de su recorrido lo más precios posible\xa0(Punto de referencia, intersección más cercana)': 'field_51', #Needed \xa0 for it to be read 
                             '¿Cuánto tiempo le tomó el recorrido caminando para legar a su lugar de destino? (Ingresar en minutos)': 'field_52',
                             'Describa la ruta que tomó para este último viaje desde su lugar de origen hasta su lugar de destino. Puede utilizar direcciones y/o lugares conocidos de referencia para facilitar la identificación...': 'field_53',
                             '¿Por qué razón utiliza la CAMINATA como medio de transporte?': 'field_54',
                             '¿Qué modo de transporte alternativo utiliza cuando por alguna razón no puede utilizar la caminata como modo de transporte?': 'field_55',
                             'Column': 'field_56',
                             '2': 'field_57',
                             '3': 'field_58',
                             '4': 'field_59',
                             '5': 'field_60',
                             '6': 'field_61',
                             '7': 'field_62',
                             '8': 'field_63',
                             '9': 'field_64',
                             '10': 'field_65',
                             '11': 'field_66',
                             '12': 'field_67'
                            },inplace=True)

# Show
missing_data

,field_1,field_2,field_3,field_4,field_5,field_6,field_7,field_8,field_9,field_11,...,field_58,field_59,field_60,field_61,field_62,field_63,field_64,field_65,field_66,field_67
140,141,7/15/24 14:47:07,7/15/24 14:53:13,anonymous,NaN,NaN,DP,6/28/2024,008,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


In [12]:
missing_data.columns

Index(['field_1', 'field_2', 'field_3', 'field_4', 'field_5', 'field_6',
       'field_7', 'field_8', 'field_9', 'field_11', 'field_13', 'field_14',
       'field_15', 'field_16', 'field_17', 'field_18', 'field_19', 'field_20',
       'field_21', 'field_22', 'field_23', 'field_24', 'field_25', 'field_26',
       'field_27', 'field_28', 'field_29', 'field_30', 'field_31', 'field_32',
       'field_33', 'field_34', 'field_35', 'field_36', 'field_37', 'field_38',
       'field_39', 'field_40', 'field_41', 'field_42', 'field_43', 'field_44',
       'field_45', 'field_46', 'field_47', 'field_48', 'field_49', 'field_50',
       'field_51', 'field_52', 'field_53', 'field_54', 'field_55', 'field_56',
       'field_57', 'field_58', 'field_59', 'field_60', 'field_61', 'field_62',
       'field_63', 'field_64', 'field_65', 'field_66', 'field_67'],
      dtype='object')

## Merge missing data

In [16]:
# Create missing routes columns to prepare to merge data
missing_data['field_10'] = '1'

# Make sure all dtypes match those in routes before filling data
for col in list(missing_data.columns):
    if col in routes.columns:
        # dtype in both gdfs
        d_type_missing = dict(missing_data.dtypes)[col]
        d_type_routes = dict(routes.dtypes)[col]
        # If different, match missing to routes before filling
        if d_type_missing != d_type_routes:
            missing_data[col] = missing_data[col].astype(d_type_routes)

# Add missing data
already_filled_cols = ['fid', 'Código', 'layer', 'path', 'Codigo_2','geometry']
for col in list(routes.columns):
    if col in already_filled_cols:
        continue
    else:
        data = missing_data[col].unique()[0]
        idx = routes.Código =='DP1V008'
        routes.loc[idx,col]= data

# Show
routes.loc[routes.Código=='DP1V008'][['field_1', 'field_2', 'field_3', 'field_4', 'field_5', 'field_6',
       'field_7', 'field_8', 'field_9', 'field_11', 'field_13', 'field_14',
       'field_15', 'field_16', 'field_17', 'field_18', 'field_19', 'field_20']]

,field_1,field_2,field_3,field_4,field_5,field_6,field_7,field_8,field_9,field_11,field_13,field_14,field_15,field_16,field_17,field_18,field_19,field_20
31,141,7/15/24 14:47:07,7/15/24 14:53:13,anonymous,NaN,NaN,DP,6/28/2024,008,TA_Providencia,Parque Rubén Darío,Masculino,50-59,NaN,Autoempleo,Pregrado,Auto - coche;,Providencia


In [17]:
# Show
routes.loc[routes.Código=='DP1V008'][['field_21', 'field_22', 'field_23', 'field_24', 'field_25', 'field_26',
       'field_27', 'field_28', 'field_29', 'field_30', 'field_31', 'field_32',
       'field_33', 'field_34', 'field_35', 'field_36', 'field_37', 'field_38']]

,field_21,field_22,field_23,field_24,field_25,field_26,field_27,field_28,field_29,field_30,field_31,field_32,field_33,field_34,field_35,field_36,field_37,field_38
31,Si,NaN,NaN,Parques;Comercial;,Cultural/Deportivo,Caminata,NaN,5 o más,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Muy importante


In [18]:
# Show
routes.loc[routes.Código=='DP1V008'][['field_39', 'field_40', 'field_41', 'field_42', 'field_43', 'field_44',
       'field_45', 'field_46', 'field_47', 'field_48', 'field_49', 'field_50','field_51',
       'field_52', 'field_53', 'field_54', 'field_55', 'field_56', 'field_57']]

,field_39,field_40,field_41,field_42,field_43,field_44,field_45,field_46,field_47,field_48,field_49,field_50,field_51,field_52,field_53,field_54,field_55,field_56,field_57
31,Muy importante,Muy importante,Importante,No es importante,Muy importante,Muy importante,Muy importante,No,NaN,Calle Álcamo,400.0,0.0,Parque Rubén Darío,8.0,"Calle Álcamo cerca del Banregio, siguiendo por...",Paseo de su mascota;,Carro particular - coche,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


In [19]:
# Show
routes.loc[routes.Código=='DP1V008'][['field_52', 'field_53', 'field_54', 'field_55', 'field_56', 'field_57',
       'field_58', 'field_59', 'field_60', 'field_61', 'field_62', 'field_63',
       'field_64', 'field_65', 'field_66', 'field_67']]

,field_52,field_53,field_54,field_55,field_56,field_57,field_58,field_59,field_60,field_61,field_62,field_63,field_64,field_65,field_66,field_67
31,8.0,"Calle Álcamo cerca del Banregio, siguiendo por...",Paseo de su mascota;,Carro particular - coche,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


In [20]:
routes.columns

Index(['fid', 'Código', 'layer', 'path', 'Codigo_2', 'field_1', 'field_2',
       'field_3', 'field_4', 'field_5', 'field_6', 'field_7', 'field_8',
       'field_9', 'field_10', 'field_11', 'field_13', 'field_14', 'field_15',
       'field_16', 'field_17', 'field_18', 'field_19', 'field_20', 'field_21',
       'field_22', 'field_23', 'field_24', 'field_25', 'field_26', 'field_27',
       'field_28', 'field_29', 'field_30', 'field_31', 'field_32', 'field_33',
       'field_34', 'field_35', 'field_36', 'field_37', 'field_38', 'field_39',
       'field_40', 'field_41', 'field_42', 'field_43', 'field_44', 'field_45',
       'field_46', 'field_47', 'field_48', 'field_49', 'field_50', 'field_51',
       'field_52', 'field_53', 'field_54', 'field_55', 'field_56', 'field_57',
       'field_58', 'field_59', 'field_60', 'field_61', 'field_62', 'field_63',
       'field_64', 'field_65', 'field_66', 'field_67', 'geometry'],
      dtype='object')

In [21]:
dict(routes.dtypes)

{'fid': dtype('int64'),
 'Código': dtype('O'),
 'layer': dtype('O'),
 'path': dtype('O'),
 'Codigo_2': dtype('O'),
 'field_1': dtype('O'),
 'field_2': dtype('O'),
 'field_3': dtype('O'),
 'field_4': dtype('O'),
 'field_5': dtype('O'),
 'field_6': dtype('O'),
 'field_7': dtype('O'),
 'field_8': dtype('O'),
 'field_9': dtype('O'),
 'field_10': dtype('O'),
 'field_11': dtype('O'),
 'field_13': dtype('O'),
 'field_14': dtype('O'),
 'field_15': dtype('O'),
 'field_16': dtype('O'),
 'field_17': dtype('O'),
 'field_18': dtype('O'),
 'field_19': dtype('O'),
 'field_20': dtype('O'),
 'field_21': dtype('O'),
 'field_22': dtype('O'),
 'field_23': dtype('O'),
 'field_24': dtype('O'),
 'field_25': dtype('O'),
 'field_26': dtype('O'),
 'field_27': dtype('O'),
 'field_28': dtype('O'),
 'field_29': dtype('O'),
 'field_30': dtype('O'),
 'field_31': dtype('O'),
 'field_32': dtype('O'),
 'field_33': dtype('O'),
 'field_34': dtype('O'),
 'field_35': dtype('O'),
 'field_36': dtype('O'),
 'field_37': dtype(

## Export filled data

In [22]:
routes.drop(columns=['fid'],inplace=True)
routes.head(2)

,Código,layer,path,Codigo_2,field_1,field_2,field_3,field_4,field_5,field_6,...,field_59,field_60,field_61,field_62,field_63,field_64,field_65,field_66,field_67,geometry
0,DP3V002,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,DP3002,252,9/19/24 18:40:07,9/19/24 18:46:58,anonymous,None,None,...,De acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,"LINESTRING (660511.087 2283608.716, 660443.453..."
1,DP3V005,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,DP3005,255,9/19/24 18:50:40,9/19/24 18:57:14,anonymous,None,None,...,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,En desacuerdo,De acuerdo,De acuerdo,"LINESTRING (660562.586 2283275.252, 660591.611..."


In [23]:
routes.to_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/preliminar_routes.gpkg")